In [2]:
import pandas as pd
import numpy as np
import string
import spacy
import multiprocessing 

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
nlp = spacy.load('en_core_web_lg')

In [4]:
dataFile = "../data/cleaned.csv"
data1 = pd.read_csv(dataFile)
data2 = pd.read_csv('../data/hate_cleaned.csv')

In [5]:
data2 = data2.drop(data2[data2['Class'] == 'neither'].index)
data2['Class'] = 1

In [ ]:
data[data.isnull().any(axis=1)]

In [6]:
data[data['Class'] == 1].tail(n=10)

NameError: name 'data' is not defined

In [7]:
ent_whitelist = set(['ADJ', 'ADV', 'PROPN', 'VERB', 'NOUN', 'INTJ', 'PRON', 'X', 'NUM'])
whitelist = set(string.ascii_lowercase + string.digits)

In [8]:
megaSpam = pd.read_csv('../data/polluters.tsv', sep='\t')
megaNonSpam = pd.read_csv('../data/legit.tsv', sep='\t')

In [9]:
megaSpam['Class'] = 1
megaNonSpam['Class'] = 0

In [10]:
data = pd.concat([data1, data2, megaSpam, megaNonSpam])
data.head()

/opt/anaconda/envs/citrus2019/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Class,FavCount,RtCount,ScreenName,Text,TweetID,Verified
0,0,0.0,8.0,Tactical_Things,RT @HouseOfTraitors: FARAGE 12 - 0 Mandelsc...,732588910078001152,1.0
1,0,0.0,0.0,kataisa,@TheBriefing2016 @HillaryClinton Can Hillary s...,732588910073778176,1.0
2,0,0.0,5423.0,Josh_Ward_,RT @HQUESTED: Harvey price just said cunt on n...,732588910061232128,1.0
3,0,0.0,0.0,networkMCI,@xonighttimexo I will put them in a Safety dep...,732588910056902656,1.0
4,0,0.0,0.0,nbharper,"you should listen, but you probably won't bc y...",732588910069620736,1.0


In [11]:
data.tail()

,Class,FavCount,RtCount,ScreenName,Text,TweetID,Verified
3246372,0,NaN,NaN,NaN,meet me on ELIMINATE pro!!,6168916131,NaN
3246373,0,NaN,NaN,NaN,my twitter,6170059145,NaN
3246374,0,NaN,NaN,NaN,exploring this thing...,6171947104,NaN
3246375,0,NaN,NaN,NaN,"learning the rap ice ice baby LOL, not very gd...",6172856187,NaN
3246376,0,NaN,NaN,NaN,"Clipse, B.o.B и J.Cole",6174743151,NaN


In [12]:
def cleanWord(word):
    word = "".join([c for c in word.lower() if c in whitelist])
    return word

def invalidWord(word):
    if (len(word) == 0 or word[0] == '@' or word == '-pron-'):
        return True
    word = cleanWord(word)
    return len(word) == 0

def cleanText(text):
    if not isinstance(text, str):
        print('Found error: {}, {}'.format(text, type(text)))
        text = str(text)
#     output = []
#     doc = nlp(text)
#     for tok in doc:
#         if tok.pos_ in ent_whitelist:
#             output.append(tok.lemma_)
    words = text.split(' ')
    output = [cleanWord(word) for word in words if not invalidWord(word)]
    return ' '.join(output)

with multiprocessing.Pool(8) as p:
#     cleanedData = p.map(cleanText, cleanData, 100)
    cleanData = p.map(cleanText, data['Text'], 100)
#     for text in data["Text"]:
#     #     print(type(text))
#         cleanData.append(cleanText(str(text)))

Found error: nan, <class 'float'>
Found error: nan, <class 'float'>
Found error: nan, <class 'float'>
Found error: nan, <class 'float'>
Found error: nan, <class 'float'>
Found error: nan, <class 'float'>
Found error: nan, <class 'float'>
Found error: nan, <class 'float'>


In [13]:
cleanData = [' '.join([x.lower() for x in y.split(' ') if len(x) > 0 and x[0] != '@']) for y in cleanData]

In [14]:
cleanData[-12312]

'combo chemothrpy radiation thrpy andor surgery in treating ppl w highrisk kidney tumors httpbitly5rdq4x'

In [ ]:
# vectorizer = CountVectorizer()
# for i in range(0, len(cleanData) - 9999, 10000):
# #     X = vectorizer.fit_transform(cleanData[i:i+10000])
    

In [ ]:


X.shape
# X.toarray().shape

In [15]:
# encoder = LabelEncoder()
# y = encoder.fit_transform(data["Class"])
y = data['Class']

In [ ]:
print(y.shape)
print(len(cleanData))

In [16]:
clean_train, clean_test, y_train, y_test = train_test_split(cleanData, y, test_size=0.25, random_state=42)
print(len(clean_test), len(y_test))
print(len(clean_train), len(y_train))
vectorizer = CountVectorizer()
X_vec = vectorizer.fit_transform(clean_train)

# classifier = GradientBoostingClassifier()
classifier = LogisticRegression()
classifier.fit(X_vec, y_train)

y_vec = vectorizer.transform(clean_test)
y_hat = classifier.predict(y_vec)


1409704 1409704
4229112 4229112


/opt/anaconda/envs/citrus2019/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda/envs/citrus2019/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
classifier = GradientBoostingClassifier()
classifier.fit(X, y)

In [ ]:
from collections import Counter
Counter(y)
# Counter(y)

In [17]:
accuracy = accuracy_score(y_test, y_hat)
accuracy

0.8267877511874834

In [ ]:
accuracy = accuracy_score(y_test, y_hat)
accuracy

# PICKLE EVERYTHING

In [18]:
import pickle
with open('thepickle_LR_beta.pkl', 'bw') as f:
#     pickle.dump([encoder, vectorizer, classifier], f)
    pickle.dump([vectorizer, classifier], f)

In [20]:
coefs = classifier.coef_[0]

In [21]:
k = 100
top_k = np.argpartition(coefs, -k)[-k:]
top_k_sorted=top_k[np.argsort(coefs[top_k])]

In [22]:
feats = vectorizer.get_feature_names()
[feats[x] for x in top_k_sorted]

['pornoru',
 'ipads',
 'httpbitlyddypiw',
 'httplinkbeecomen25t',
 'wwwvaldecyalvesblogspotcom',
 'httpbitlyeziread',
 'chatroulette',
 'httpbitlysr9wm',
 'luvmystripes',
 'asfc',
 'soshified',
 'httplovinmysistascom',
 'httpwwwyoutubecomwatchv674o8mmurfc',
 'httpwwwpulsepowernowcom',
 'httpbitlycu9ead',
 'httpbitly3tmcrn',
 'dateyocom',
 'dubli',
 'httpappsfacebookcomiplpredict',
 'papiroflexia',
 'httpisgda3eeu',
 'httpwwwplaymobsterworldcomplatformtwittersourceonelinefight',
 'freegiladshalit',
 'heartfeltquotes',
 'cbb7',
 'httpbitly4i1wfq',
 'jpg',
 'httpwwwjinxbeatzcom',
 'thatswhatshesaid',
 'httpstoresebaycomhistoricdvds',
 'efrcm',
 'upskirt',
 'httpbitlyb5e7kt',
 'httpagd7d4b29',
 'theillblogcom',
 'wwwmodlowarvaicom',
 'sbchocolate',
 'algarve',
 'cuteasxkiki',
 'nexus',
 'httpdrfuzzcom',
 'httpwwwelegantvisionofbeautycom',
 'httpwwwtinychatcomtonydofat',
 'httpbitly8yjfas',
 'ihatequotes',
 'iconfess',
 'stonerwhen',
 'wwwchchjobsconz',
 'httpbitlyard1gu',
 'untranslatable'